In [1]:
import warnings
warnings.filterwarnings('ignore')
from  Model_test import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling
from sklearn.decomposition import PCA

In [2]:
pd.options.display.max_columns = 90

In [3]:
df = pd.read_csv('dataset_diabetes/diabetic_data.csv')

In [4]:
df.head(25)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,?,?,31,6,16,0,0,0,414,411,250,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
6,55842,84259809,Caucasian,Male,[60-70),?,3,1,2,4,?,?,70,1,21,0,0,0,414,411,V45,7,None,None,Steady,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,63768,114882984,Caucasian,Male,[70-80),?,1,1,7,5,?,?,73,0,12,0,0,0,428,492,250,8,None,None,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,>30
8,12522,48330783,Caucasian,Female,[80-90),?,2,1,4,13,?,?,68,2,28,0,0,0,398,427,38,8,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,15738,63555939,Caucasian,Female,[90-100),?,3,3,4,12,?,InternalMedicine,33,3,18,0,0,0,434,198,486,8,None,None,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [5]:
# pandas_profiling.ProfileReport(df)

## Cleaning some data


In [6]:
# Drop becuase all values are the same
df.drop(columns = ['patient_nbr','citoglipton','weight','examide','encounter_id'],inplace=True)

# weight, drop coz 97% missing
# citoglipton, and examide are constant values
# encounter_id is just an id

In [7]:
# race, missing set to other 2%
df.race.replace('?', 'Other',inplace=True)


In [8]:
df.drop(columns = ['diag_1', 'diag_2','diag_3','payer_code'],inplace=True)


In [9]:
# convert diabetes_med from Yes or No to 1 or 0
# convert readmission from No, <30, >30 to 0, 1, 2


In [10]:
# convert change from No, Ch to 0, 1
df.change = np.where(df.change=='No',0,1)

In [11]:
# convert diabetes_med from Yes or No to 1 or 0
df.diabetesMed = np.where(df.diabetesMed=='Yes',1,0)

In [12]:
df.drop('medical_specialty', axis=1, inplace=True)
# 49,000 are ?

In [13]:
df.max_glu_serum.unique()

array(['None', '>300', 'Norm', '>200'], dtype=object)

In [14]:
df.readmitted = np.where(df.readmitted=='NO', 0, df.readmitted )
df.readmitted = np.where(df.readmitted=='<30', 1,  df.readmitted )
df.readmitted = np.where(df.readmitted=='>30', 2,  df.readmitted )

In [15]:
dummy_df = pd.get_dummies(df,drop_first=True)

In [16]:
dummy_df.drop(columns=['readmitted_1', 'readmitted_2','gender_Unknown/Invalid'],
              inplace=True)

In [17]:
X = dummy_df.copy()
y = df.readmitted.astype(int)

In [18]:
col_names = X.columns
col_names_list = []
for i, col_name in enumerate(col_names):
    c1 = col_name.replace('[','')
    c1 = c1.replace('>','')
    col_names_list.append(c1)
X.columns = col_names_list
    

In [19]:
# Save X, y to a file for faster loading

## Train Test Split

In [20]:
X_train, X_test, y_train, y_test = scale_train_test_split(X,y)

In [ ]:
X.shape

## Base Model

In [21]:
models, model_names = train_model(X_train, y_train)

In [22]:
y_train_predicts = predict_all(X_train, y_train, models)

In [23]:
y_test_predicts = predict_all(X_test, y_test, models)

In [24]:

print('--- Train data ---')
for i,y_train_pred in enumerate(y_train_predicts):
    display_acc_and_f1_score(y_train, y_train_pred, model_names[i])
print('\n\n--- Test data ---')
for i,y_test_pred in enumerate(y_test_predicts):
    display_acc_and_f1_score(y_test, y_test_pred, model_names[i])

--- Train data ---
Model: AdaBoost
Accuracy: 0.5822152926995441
F1-Score: 0.5822152926995441
Model: GradientBoost
Accuracy: 0.5918583931659767
F1-Score: 0.5918583931659767
Model: XGBoost
Accuracy: 0.5884518631098999
F1-Score: 0.5884518631098999
Model: Logistic Regresion
Accuracy: 0.5715764372936429
F1-Score: 0.5715764372936429
Model: DecisionTree_gini
Accuracy: 0.5896834547455584
F1-Score: 0.5896834547455584
Model: DecisionTree_entropy
Accuracy: 0.5867617001205387
F1-Score: 0.5867617001205387


--- Test data ---
Model: AdaBoost
Accuracy: 0.5793962738778398
F1-Score: 0.5793962738778398
Model: GradientBoost
Accuracy: 0.583994968948982
F1-Score: 0.583994968948982
Model: XGBoost
Accuracy: 0.5828158163666378
F1-Score: 0.5828158163666378
Model: Logistic Regresion
Accuracy: 0.5681550192594922
F1-Score: 0.5681550192594922
Model: DecisionTree_gini
Accuracy: 0.5720462227812279
F1-Score: 0.5720462227812279
Model: DecisionTree_entropy
Accuracy: 0.5714566464900558
F1-Score: 0.5714566464900558


In [25]:
# Decision Tree overfitting because we did not limit number of splits yet

## Iteration #2

In [26]:
from imblearn.over_sampling import SMOTE
from collections import Counter

sm = SMOTE(random_state=109)
X_new_train, y_new_train = sm.fit_sample(X_train, y_train)



In [27]:
# print(np.unique(y_new, return_counts=True))
# np.unique(y, return_counts=True)

In [33]:
type(X_new_train), type(y_new_train)

(numpy.ndarray, numpy.ndarray)

In [34]:
type(X_test), type(y_test)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [28]:
models, model_names = train_model(X_new_train, y_new_train)

In [ ]:
y_new_train_predicts = predict_all(X_new_train, y_new_train, models)

In [35]:
y_new_test_predicts = predict_all(np.array(X_test), np.array(y_test), models)

In [36]:

print('--- Train data ---')
for i, y_new_train_pred in enumerate(y_new_train_predicts):
    display_acc_and_f1_score(y_new_train, y_new_train_pred, model_names[i])
print('\n\n--- Test data ---')
for i,y_new_test_pred in enumerate(y_new_test_predicts):
    display_acc_and_f1_score(y_test, y_new_test_pred, model_names[i])

--- Train data ---
Model: AdaBoost
Accuracy: 0.5685736042253977
F1-Score: 0.5685736042253977
Model: GradientBoost
Accuracy: 0.6076358510741713
F1-Score: 0.6076358510741713
Model: XGBoost
Accuracy: 0.6047843556592463
F1-Score: 0.6047843556592463
Model: Logistic Regresion
Accuracy: 0.5965133987881145
F1-Score: 0.5965133987881145
Model: DecisionTree_gini
Accuracy: 0.5543242279900198
F1-Score: 0.5543242279900198
Model: DecisionTree_entropy
Accuracy: 0.5505897410971776
F1-Score: 0.5505897410971776


--- Test data ---
Model: AdaBoost
Accuracy: 0.5266488483609779
F1-Score: 0.5266488483609779
Model: GradientBoost
Accuracy: 0.5736970363965097
F1-Score: 0.5736970363965097
Model: XGBoost
Accuracy: 0.5721641380394623
F1-Score: 0.5721641380394623
Model: Logistic Regresion
Accuracy: 0.5592327647197547
F1-Score: 0.5592327647197547
Model: DecisionTree_gini
Accuracy: 0.5299897806776197
F1-Score: 0.5299897806776197
Model: DecisionTree_entropy
Accuracy: 0.5202814244163195
F1-Score: 0.5202814244163195


## Iteration #3

In [43]:
len(X.columns)

81

In [44]:
from sklearn.decomposition import PCA
import numpy as np

pca_1 = PCA(n_components=20)
pca_2 = PCA(n_components=40)
pca_3 = PCA(n_components=60)

principalComponents = pca_1.fit_transform(X)
principalComponents = pca_2.fit_transform(X)
principalComponents = pca_3.fit_transform(X)

print(np.sum(pca_1.explained_variance_ratio_))
print(np.sum(pca_2.explained_variance_ratio_))
print(np.sum(pca_3.explained_variance_ratio_))





# print(pca_1.explained_variance_ratio_)
# print(pca_2.explained_variance_ratio_)
# print(pca_3.explained_variance_ratio_)
# print(np.sum(pca_2.explained_variance_ratio_))
# print(np.sum(pca_3.explained_variance_ratio_))

0.9975261382221873
0.9998231123227077
0.9999964313060066


In [52]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
print(np.sum(pca.explained_variance_ratio_))

0.8743319127661378


In [53]:
pca.components_

array([[-9.99392985e-03,  8.63279563e-03,  9.60676788e-03,
         5.20607116e-02,  9.89616255e-01,  6.84145340e-03,
         1.31572175e-01, -3.15784000e-04, -6.62887948e-05,
         2.73345229e-03,  1.63327551e-02,  1.93526432e-03,
         9.13577066e-04, -3.61542182e-05, -4.24568988e-04,
        -2.99916418e-05,  6.12916483e-05, -1.11788522e-04,
        -1.98659292e-05, -2.65357509e-05, -3.51093719e-05,
        -1.01682124e-04, -1.83639293e-04, -1.93186954e-04,
         8.38307093e-05,  3.97414704e-04,  9.30018121e-05,
        -1.88539035e-04,  1.81595265e-03, -1.09102898e-03,
         2.39462691e-03, -4.75473006e-03,  1.37591400e-03,
         8.67975531e-04, -1.00112552e-03,  9.86013202e-05,
        -8.57297716e-05,  4.75563993e-05,  3.01771626e-05,
         2.06578999e-05, -2.40870885e-05,  3.00163166e-06,
         2.84351897e-06, -2.51114257e-06,  7.39840026e-07,
        -2.11627283e-05, -7.71002925e-05,  6.34503385e-05,
         6.91742263e-07, -2.74194894e-04,  4.51215411e-0

In [54]:
pca.mean_

array([2.02400605e+00, 3.71564177e+00, 5.75443665e+00, 4.39598687e+00,
       4.30956410e+01, 1.33973036e+00, 1.60218442e+01, 3.69357153e-01,
       1.97836212e-01, 6.35565906e-01, 7.42260676e+00, 4.61951929e-01,
       7.70031248e-01, 6.29876383e-03, 7.47784132e-01, 2.00165085e-02,
       3.71342099e-02, 4.62384293e-01, 6.79008706e-03, 1.62824519e-02,
       3.70949040e-02, 9.51693100e-02, 1.69565474e-01, 2.20928404e-01,
       2.56156280e-01, 1.68985712e-01, 2.74453157e-02, 1.24206513e-02,
       9.47467720e-01, 2.55193287e-02, 8.07342334e-02, 8.32773225e-01,
       4.90340585e-02, 8.03588625e-01, 1.80276320e-01, 1.04848378e-02,
       9.84877071e-01, 1.35998271e-02, 1.08091111e-03, 9.93091995e-01,
       6.56407838e-03, 2.35835151e-04, 9.99154924e-01, 7.76290706e-04,
       5.89587878e-05, 9.48990822e-01, 4.58895898e-02, 3.21325394e-03,
       9.82646463e-06, 8.75341470e-01, 1.11589332e-01, 7.56637777e-03,
       8.95348152e-01, 9.11306330e-02, 7.97908928e-03, 2.26008687e-04,
      

In [62]:
pca_df = pd.DataFrame(principalComponents, columns= ['PCA1', 'PCA2'])

In [63]:
X_train, X_test, y_train, y_test = scale_train_test_split(pca_df,y)

In [68]:
models, model_names = train_model(X_train, y_train)

In [69]:
y_train_predicts = predict_all(X_train, y_train, models)
y_test_predicts = predict_all(X_test, y_test, models)

In [70]:
print('--- Train data ---')
for i, y_new_train_pred in enumerate(y_new_train_predicts):
    display_acc_and_f1_score(y_new_train, y_new_train_pred, model_names[i])
print('\n\n--- Test data ---')
for i,y_new_test_pred in enumerate(y_new_test_predicts):
    display_acc_and_f1_score(y_test, y_new_test_pred, model_names[i])

--- Train data ---
Model: AdaBoost
Accuracy: 0.5685736042253977
F1-Score: 0.5685736042253977
Model: GradientBoost
Accuracy: 0.6076358510741713
F1-Score: 0.6076358510741713
Model: XGBoost
Accuracy: 0.6047843556592463
F1-Score: 0.6047843556592463
Model: Logistic Regresion
Accuracy: 0.5965133987881145
F1-Score: 0.5965133987881145
Model: DecisionTree_gini
Accuracy: 0.5543242279900198
F1-Score: 0.5543242279900198
Model: DecisionTree_entropy
Accuracy: 0.5505897410971776
F1-Score: 0.5505897410971776


--- Test data ---
Model: AdaBoost
Accuracy: 0.5266488483609779
F1-Score: 0.5266488483609779
Model: GradientBoost
Accuracy: 0.5736970363965097
F1-Score: 0.5736970363965097
Model: XGBoost
Accuracy: 0.5721641380394623
F1-Score: 0.5721641380394623
Model: Logistic Regresion
Accuracy: 0.5592327647197547
F1-Score: 0.5592327647197547
Model: DecisionTree_gini
Accuracy: 0.5299897806776197
F1-Score: 0.5299897806776197
Model: DecisionTree_entropy
Accuracy: 0.5202814244163195
F1-Score: 0.5202814244163195


## Columns to inspect

    - Admission_type_id
    - discharge_disposition_id
    - admission_source_id
    - number_of_lab_procedures
    